In [2]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import pandas as pd
from datasets import Dataset
import torch

In [3]:
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("RajeevanL/distilled_XLMRoberta_153_v2")
model = AutoModelForQuestionAnswering.from_pretrained("RajeevanL/distilled_XLMRoberta_153_v2")

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "TamilSQUAD.xlsx"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uom200501p/tamilsquad",
  file_path,
)

/tmp/ipykernel_75/2304354973.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [5]:
df

,Column1,Column2,Column3
0,Question,Context,Answer
1,பெரும்பான்மையான எஸ்டோனியர்கள் ஜெர்மானியர்களை எ...,சூழல்: ஆரம்பத்தில் ஜேர்மனியர்கள் சோவியத் ஒன்றி...,விடுதலை செய்பவர்கள்.
2,ஜப்பான் கையகப்படுத்துவதற்கு முன்பு இந்தோசீனாவை...,"சூழல்: செப்டம்பர் 1940 இல், ஜப்பான் இந்தோசீனாவ...",விச்சி பிரான்ஸ்.
3,ஐரோப்பாவில் சீர்திருத்தத்திற்கு தலைமை தாங்கியவ...,சூழல்: ஐரோப்பாவில் சீர்திருத்தம் அதிகாரப்பூர்வ...,மார்ட்டின் லூதர்.
4,IMAX திரையரங்குகளில் திரையிடப்பட்ட முதல் ஜேம்ஸ...,சூழல்: ஸ்பெக்டர் 26 அக்டோபர் 2015 அன்று ஐக்கிய...,ஸ்கைஃபால்.
...,...,...,...
77969,எம்பயர் ஸ்டேட் கட்டிடம் எப்போது கட்டப்பட்டது?,சூழல்: 1916 மண்டலத் தீர்மானம் புதிய கட்டிடங்கள...,1931.
77970,"புதிய நேச நாட்டு சமாதான திட்டத்தில், பிரான்ஸ் ...","சூழல்: நெப்போலியன், போரில் வெற்றி பெறுவார் என்...",பெல்ஜியம்.
77971,2006 FIBA ​​உலக சாம்பியன்ஷிப்பில் கிரேக்க கூடை...,சூழல்: கிரேக்க தேசிய கூடைப்பந்து அணியானது பல த...,அணி அமெரிக்கா.
77972,பட்டியலில் பிரேசில் இருந்ததா?,சூழல்: Maplecroft Child Labour Index 2012 கணக்...,பிரேசில் 54வது.


In [6]:
df = df.rename(columns={"Column1": "question", "Column2": "context", "Column3": "answer"})

In [7]:
df = df.iloc[1:].reset_index(drop=True)
df

,question,context,answer
0,பெரும்பான்மையான எஸ்டோனியர்கள் ஜெர்மானியர்களை எ...,சூழல்: ஆரம்பத்தில் ஜேர்மனியர்கள் சோவியத் ஒன்றி...,விடுதலை செய்பவர்கள்.
1,ஜப்பான் கையகப்படுத்துவதற்கு முன்பு இந்தோசீனாவை...,"சூழல்: செப்டம்பர் 1940 இல், ஜப்பான் இந்தோசீனாவ...",விச்சி பிரான்ஸ்.
2,ஐரோப்பாவில் சீர்திருத்தத்திற்கு தலைமை தாங்கியவ...,சூழல்: ஐரோப்பாவில் சீர்திருத்தம் அதிகாரப்பூர்வ...,மார்ட்டின் லூதர்.
3,IMAX திரையரங்குகளில் திரையிடப்பட்ட முதல் ஜேம்ஸ...,சூழல்: ஸ்பெக்டர் 26 அக்டோபர் 2015 அன்று ஐக்கிய...,ஸ்கைஃபால்.
4,சவுத்தாம்ப்டனில் என்ன வகையான காலநிலை உள்ளது?,"சூழல்: UK இன் மற்ற பகுதிகளைப் போலவே, சவுத்தாம்...",கடல் சார்ந்த.
...,...,...,...
77968,எம்பயர் ஸ்டேட் கட்டிடம் எப்போது கட்டப்பட்டது?,சூழல்: 1916 மண்டலத் தீர்மானம் புதிய கட்டிடங்கள...,1931.
77969,"புதிய நேச நாட்டு சமாதான திட்டத்தில், பிரான்ஸ் ...","சூழல்: நெப்போலியன், போரில் வெற்றி பெறுவார் என்...",பெல்ஜியம்.
77970,2006 FIBA ​​உலக சாம்பியன்ஷிப்பில் கிரேக்க கூடை...,சூழல்: கிரேக்க தேசிய கூடைப்பந்து அணியானது பல த...,அணி அமெரிக்கா.
77971,பட்டியலில் பிரேசில் இருந்ததா?,சூழல்: Maplecroft Child Labour Index 2012 கணக்...,பிரேசில் 54வது.


In [8]:
# Drop rows with missing context or answer
df.dropna(subset=["context", "answer"], inplace=True)

# Ensure context and answer are strings
df["context"] = df["context"].astype(str)
df["answer"] = df["answer"].astype(str)

In [9]:
df

,question,context,answer
0,பெரும்பான்மையான எஸ்டோனியர்கள் ஜெர்மானியர்களை எ...,சூழல்: ஆரம்பத்தில் ஜேர்மனியர்கள் சோவியத் ஒன்றி...,விடுதலை செய்பவர்கள்.
1,ஜப்பான் கையகப்படுத்துவதற்கு முன்பு இந்தோசீனாவை...,"சூழல்: செப்டம்பர் 1940 இல், ஜப்பான் இந்தோசீனாவ...",விச்சி பிரான்ஸ்.
2,ஐரோப்பாவில் சீர்திருத்தத்திற்கு தலைமை தாங்கியவ...,சூழல்: ஐரோப்பாவில் சீர்திருத்தம் அதிகாரப்பூர்வ...,மார்ட்டின் லூதர்.
3,IMAX திரையரங்குகளில் திரையிடப்பட்ட முதல் ஜேம்ஸ...,சூழல்: ஸ்பெக்டர் 26 அக்டோபர் 2015 அன்று ஐக்கிய...,ஸ்கைஃபால்.
4,சவுத்தாம்ப்டனில் என்ன வகையான காலநிலை உள்ளது?,"சூழல்: UK இன் மற்ற பகுதிகளைப் போலவே, சவுத்தாம்...",கடல் சார்ந்த.
...,...,...,...
77968,எம்பயர் ஸ்டேட் கட்டிடம் எப்போது கட்டப்பட்டது?,சூழல்: 1916 மண்டலத் தீர்மானம் புதிய கட்டிடங்கள...,1931.
77969,"புதிய நேச நாட்டு சமாதான திட்டத்தில், பிரான்ஸ் ...","சூழல்: நெப்போலியன், போரில் வெற்றி பெறுவார் என்...",பெல்ஜியம்.
77970,2006 FIBA ​​உலக சாம்பியன்ஷிப்பில் கிரேக்க கூடை...,சூழல்: கிரேக்க தேசிய கூடைப்பந்து அணியானது பல த...,அணி அமெரிக்கா.
77971,பட்டியலில் பிரேசில் இருந்ததா?,சூழல்: Maplecroft Child Labour Index 2012 கணக்...,பிரேசில் 54வது.


In [10]:
def fuzzy_match_answer(row, threshold=0.75):
    context = str(row["context"]) if pd.notnull(row["context"]) else ""
    answer = str(row["answer"]) if pd.notnull(row["answer"]) else ""

    answer_len = len(answer)
    best_match_start = -1
    best_match_len = 0

    # Sliding window to check partial matches
    for i in range(answer_len):
        for j in range(i + int(threshold * answer_len), answer_len + 1):
            substring = answer[i:j]
            start_idx = context.find(substring)
            if start_idx != -1 and (j - i) > best_match_len:
                best_match_len = j - i
                best_match_start = start_idx

    if best_match_len >= threshold * answer_len:
        return {
            "text": [answer],
            "answer_start": [best_match_start]
        }
    else:
        return {
            "text": [answer],
            "answer_start": [0]  # default when not found
        }

# Apply to the dataframe
df["answers"] = df.apply(fuzzy_match_answer, axis=1)


In [12]:
df['answers']

0        {'text': ['விடுதலை செய்பவர்கள்.'], 'answer_sta...
1        {'text': ['விச்சி பிரான்ஸ்.'], 'answer_start':...
2        {'text': ['மார்ட்டின் லூதர்.'], 'answer_start'...
3          {'text': ['ஸ்கைஃபால்.'], 'answer_start': [245]}
4         {'text': ['கடல் சார்ந்த.'], 'answer_start': [0]}
                               ...                        
77968           {'text': ['1931.'], 'answer_start': [260]}
77969        {'text': ['பெல்ஜியம்.'], 'answer_start': [0]}
77970    {'text': ['அணி அமெரிக்கா.'], 'answer_start': [...
77971    {'text': ['பிரேசில் 54வது.'], 'answer_start': ...
77972    {'text': ['ஃபோர்ட் ரோஸ்க்ரான்ஸ்.'], 'answer_st...
Name: answers, Length: 77570, dtype: object

In [13]:
# Count number of rows where answer_start is 0
num_missing_answers = df["answers"].apply(lambda x: x["answer_start"][0] == 0).sum()

print(f"Number of rows with answer_start = 0: {num_missing_answers}")

Number of rows with answer_start = 0: 26224


In [14]:
df = df[df["answers"].apply(lambda x: x["answer_start"][0] != 0)].reset_index(drop=True)

In [15]:
df

,question,context,answer,answers
0,ஜப்பான் கையகப்படுத்துவதற்கு முன்பு இந்தோசீனாவை...,"சூழல்: செப்டம்பர் 1940 இல், ஜப்பான் இந்தோசீனாவ...",விச்சி பிரான்ஸ்.,"{'text': ['விச்சி பிரான்ஸ்.'], 'answer_start':..."
1,ஐரோப்பாவில் சீர்திருத்தத்திற்கு தலைமை தாங்கியவ...,சூழல்: ஐரோப்பாவில் சீர்திருத்தம் அதிகாரப்பூர்வ...,மார்ட்டின் லூதர்.,"{'text': ['மார்ட்டின் லூதர்.'], 'answer_start'..."
2,IMAX திரையரங்குகளில் திரையிடப்பட்ட முதல் ஜேம்ஸ...,சூழல்: ஸ்பெக்டர் 26 அக்டோபர் 2015 அன்று ஐக்கிய...,ஸ்கைஃபால்.,"{'text': ['ஸ்கைஃபால்.'], 'answer_start': [245]}"
3,எந்த வகையான தாவரங்கள் ஒருபோதும் இதய மரத்தை உரு...,சூழல்: ஹார்ட்வுட் பெரும்பாலும் உயிருள்ள சப்வுட...,மரத்தாலான.,"{'text': ['மரத்தாலான.'], 'answer_start': [328]}"
4,எந்த குழுக்கள் மேட்ரிக்ஸ் பெருக்கத்துடன் மெட்ர...,சூழல்: மேட்ரிக்ஸ் குழுக்கள் மேட்ரிக்ஸ் பெருக்க...,மேட்ரிக்ஸ் குழுக்கள்.,"{'text': ['மேட்ரிக்ஸ் குழுக்கள்.'], 'answer_st..."
...,...,...,...,...
51341,1924 தேர்தலில் எத்தனை எம்.பி.,சூழல்: 1923 பொதுத் தேர்தல் கன்சர்வேடிவ்களின் ப...,191.,"{'text': ['191.'], 'answer_start': [463]}"
51342,எம்பயர் ஸ்டேட் கட்டிடம் எப்போது கட்டப்பட்டது?,சூழல்: 1916 மண்டலத் தீர்மானம் புதிய கட்டிடங்கள...,1931.,"{'text': ['1931.'], 'answer_start': [260]}"
51343,2006 FIBA ​​உலக சாம்பியன்ஷிப்பில் கிரேக்க கூடை...,சூழல்: கிரேக்க தேசிய கூடைப்பந்து அணியானது பல த...,அணி அமெரிக்கா.,"{'text': ['அணி அமெரிக்கா.'], 'answer_start': [..."
51344,பட்டியலில் பிரேசில் இருந்ததா?,சூழல்: Maplecroft Child Labour Index 2012 கணக்...,பிரேசில் 54வது.,"{'text': ['பிரேசில் 54வது.'], 'answer_start': ..."


In [16]:
dataset = Dataset.from_pandas(df)

In [17]:
def preprocess_function(examples):
    questions = [q.strip() if isinstance(q, str) else "" for q in examples["question"]]
    contexts = [c if isinstance(c, str) else "" for c in examples["context"]]
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        context = contexts[i]
        answer_data = examples["answers"][i]
        
        if not answer_data or not answer_data.get("text") or not answer_data.get("answer_start"):
            start_positions.append(0)
            end_positions.append(0)
            continue

        answer_text = answer_data["text"][0]
        start_char = answer_data["answer_start"][0]
        end_char = start_char + len(answer_text)
        sequence_ids = inputs.sequence_ids(i)

        # Find context token start and end
        context_start = None
        context_end = None
        for idx, seq_id in enumerate(sequence_ids):
            if seq_id == 1 and context_start is None:
                context_start = idx
            if seq_id == 1:
                context_end = idx

        if context_start is None or context_end is None:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Skip if answer not in span
        if offsets[context_start][0] > end_char or offsets[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Token start index
        idx = context_start
        while idx <= context_end and offsets[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        # Token end index
        idx = context_end
        while idx >= context_start and offsets[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [18]:
# Tokenize
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/51346 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 51346
})

In [20]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 41076
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10270
    })
})

In [24]:
# Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[], 
)

In [25]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

/tmp/ipykernel_75/1566634480.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.120700,4.074160
2,4.001800,4.020981
3,3.937200,4.010736
4,3.904000,3.999824
5,3.881700,3.995830


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=6420, training_loss=3.979870130117066, metrics={'train_runtime': 4054.7932, 'train_samples_per_second': 50.651, 'train_steps_per_second': 1.583, 'total_flos': 1.193452193977344e+16, 'train_loss': 3.979870130117066, 'epoch': 5.0})

In [26]:
# Save model and tokenizer locally in Kaggle working directory
model.save_pretrained("/kaggle/working/fine_tuned_tamil_qa_model")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_tamil_qa_model")

('/kaggle/working/fine_tuned_tamil_qa_model/tokenizer_config.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/special_tokens_map.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/sentencepiece.bpe.model',
 '/kaggle/working/fine_tuned_tamil_qa_model/added_tokens.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/tokenizer.json')

In [ ]:
# Push to Hugging Face Hub
from huggingface_hub import login
import os

# Manually set your Hugging Face token here as a string
hf_token = "YOUR_HUGGING_FACE_TOKEN"
login(token=hf_token)

In [28]:
model.push_to_hub("RajeevanL/distilled_XLMRoberta_153_v3")
tokenizer.push_to_hub("RajeevanL/distilled_XLMRoberta_153_v3")

model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RajeevanL/distilled_XLMRoberta_153_v3/commit/472efec3b58b6f988eddc90510d740b7724d9843', commit_message='Upload tokenizer', commit_description='', oid='472efec3b58b6f988eddc90510d740b7724d9843', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RajeevanL/distilled_XLMRoberta_153_v3', endpoint='https://huggingface.co', repo_type='model', repo_id='RajeevanL/distilled_XLMRoberta_153_v3'), pr_revision=None, pr_num=None)